In [12]:
import array, io, json, collections
import numpy as np
from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_absolute_error

In [50]:
class SentimentClassifier:

    def __init__(self, cutoff=10000):
        self.wordVecs = self.readGlove()
        self.reviews, self.features, self.targets = self.readYelpReviews(cutoff)
        self.model = GaussianNB()
        self.n = len(self.targets)
        self.d = len(self.features[0])

    # Adapted from maciejkula
    # Read in the GloVe vector data
    def readGlove(self):
        dct = {}
        with io.open('vectors/glove.6B.300d.txt', 'r', encoding='utf-8') as savefile:
            for i, line in enumerate(savefile):
                if i % 100 == 0:
                    print i
                tokens = line.split(' ')

                word = tokens[0]
                entries = tokens[1:]

                dct[word] = np.array([float(x) for x in entries])
        return dct

    # Read in the Yelp review data
    def readYelpReviews(self, cutoff=None):
        reviews, features, targets = [], [], []
        with io.open('data/yelp_academic_dataset_review.json', 'r', encoding='utf-8') as savefile:
            for i, line in enumerate(savefile):
                if i%100 == 0:
                    print i
                if cutoff and i > cutoff:
                    break
                review = json.loads(line)
                targets.append(review['stars'])
                features.append(self.generateFeatures(review))
                reviews.append(review)
        return reviews, features, targets

    def generateFeatures(self, review):
        text = review['text']
        wordVecs = [self.wordVecs[token] for token in text.split() if token in self.wordVecs]
        featureVec = np.mean(wordVecs, axis=0) if wordVecs else np.zeros(len(self.wordVecs['the']))
        return featureVec

    def crossValidate(self, k=10):
        for i in range(k):
            lower, upper = i*(self.n/k), (i+1)*(self.n/k)
            xTrain, yTrain = self.features[lower:upper], self.targets[lower:upper]
            xTest, yTest = self.features[lower:upper], self.targets[lower:upper]
            self.model.fit(xTrain, yTrain)
            prediction_log_probas = np.array(self.model.predict_log_proba(xTest))
            print prediction_log_proba[0], yTest[0]
            scores.append(-prediction_log_probas[range(len(prediction_log_probas)), yTest - np.ones(len(yTest), dtype=int)])
        print np.mean(scores)
    
    def train(self, start, end):
        self.model.fit(self.features[start:end], self.targets[start:end])

    def test(self, start, end):
        predictions = self.model.predict(self.features[start:end])
        print mean_absolute_error(self.ratings[start:end], predictions)

In [51]:
clf = SentimentClassifier()
clf.crossValidate()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

NameError: global name 'prediction_log_proba' is not defined

In [66]:
k=10
scores = []
for i in range(k):
    lower, upper = i*(clf.n/k), (i+1)*(clf.n/k)
    xTrain, yTrain = clf.features[:lower] + clf.features[upper:], clf.targets[:lower] + clf.targets[upper:]
    xTest, yTest = clf.features[lower:upper], clf.targets[lower:upper]
    clf.model.fit(xTrain, yTrain)
    prediction_log_probas = np.array(clf.model.predict_log_proba(xTest))
    print -prediction_log_probas[range(len(prediction_log_probas)), yTest - np.ones(len(yTest), dtype=int)]
    ce = np.mean(-prediction_log_probas[range(len(prediction_log_probas)), yTest - np.ones(len(yTest), dtype=int)])
    scores.append(ce)
print scores

[  2.18720012e+01   3.98560247e+00   2.39133457e+01   1.73189678e+01
   1.76285082e+01   4.86874716e-02   1.71478564e+01   1.06326324e+01
   7.48259481e-08   1.78836533e+00   1.57865123e-03   8.80544486e+00
   8.58507686e+01   4.71446845e-05   1.02784367e+00   2.22295252e+01
   2.00017049e+01   3.83515593e+01   1.00508819e+01   4.15448493e+01
   2.72944259e+01   2.63836161e+01   5.39205825e+01  -0.00000000e+00
   5.51020719e+01   6.98903535e-02   2.59774424e-11   2.46421211e+01
   4.33869167e+01  -0.00000000e+00   7.47503294e+01   2.70865874e+01
   3.45541570e+01   3.04268478e+01   2.52508783e+01   3.42320488e-01
   5.92029855e+00  -0.00000000e+00   1.88377889e+01   2.19010073e+01
   4.19580212e+00   4.09035835e+01   5.55265498e+01   1.47223794e+01
   2.56411511e-07   1.61478556e-06  -0.00000000e+00   3.07428888e+01
   3.79007608e-04   2.16730715e-02   1.62776044e+01   1.51342609e+01
   2.74957166e+01   3.33102435e-11   7.54421257e-04   2.46496771e+01
   1.80330235e+01   2.91690801e-02